In [9]:
from random import randint, uniform
from util import generate_problem, verify_witness

In [6]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def walk_gene(gene, T):
    # TODO: how to do this?
    
    # attempt 1: find smallest diff for constraint to be satisfied
    failed = verify_witness(gene, T)
    for cons in failed:
        i, j = cons[i], cons[j]
        intervals = cons[intervals]
        
        # oof
        
    
    
    
    # attempt 2: find variable which breaks most constraints, then?

def walk_tcsp(T, r, max_iterations, max_flips):
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T, r)
        gene_failed = verify_witness(gene, T)
        if not best_gene or len(gene_failed) < len(best_gene_failed):
            best_gene = gene
            best_gene_failed = gene_failed
        
        if len(gene_failed) == 0:
            break
    
        for j in range(max_flips):
            gene = walk_gene(gene, T)
            gene_failed = verify_witness(gene, T)
            
            if len(gene_failed) == 0:
                break
                
        # necessary for double break after flips loop
        if len(gene_failed) == 0: break 
    
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)